# 🏦 Notebook 01: Setup del Ambiente

## Proyecto End-to-End: CDC, SCD y Liquid Clustering

---

### Objetivos de este notebook:
1. Configurar el ambiente de Databricks
2. Crear la base de datos del proyecto
3. Crear las tablas Bronze con CDC y Liquid Clustering habilitado
4. Generar datos sintéticos para el caso financiero

### Requisitos:
- Azure Databricks Runtime 15.2+ LTS (recomendado para Liquid Clustering GA)
- Cluster configurado con autoscaling

---

## 1️⃣ Verificación del Ambiente

In [ ]:
# Verificar versión de Spark y Databricks Runtime
print(f"Spark Version: {spark.version}")
print(f"Databricks Runtime: {spark.conf.get('spark.databricks.clusterUsageTags.sparkVersion', 'N/A')}")

# Verificar si Liquid Clustering está disponible (DBR 13.3+)
try:
    spark.sql("SELECT 1").show()
    print("✅ Conexión a Spark exitosa")
except Exception as e:
    print(f"❌ Error: {e}")

## 2️⃣ Configuración de Variables del Proyecto

In [ ]:
# ============================================
# CONFIGURACIÓN DEL PROYECTO
# ============================================

# Nombre de la base de datos
DATABASE_NAME = "financial_lakehouse"

# Ubicación de almacenamiento (ajustar según tu configuración)
# Para Unity Catalog, usar: catalog.schema
# Para Hive Metastore, usar path en DBFS o ADLS
STORAGE_PATH = f"/mnt/delta/{DATABASE_NAME}"

# Configuración de tablas
TABLES = {
    "bronze": {
        "clientes": "bronze_clientes",
        "cuentas": "bronze_cuentas",
        "transacciones": "bronze_transacciones"
    },
    "silver": {
        "clientes": "silver_dim_clientes",
        "cuentas": "silver_dim_cuentas",
        "transacciones": "silver_fact_transacciones"
    },
    "gold": {
        "resumen_cliente": "gold_resumen_cliente",
        "metricas_diarias": "gold_metricas_diarias",
        "analisis_riesgo": "gold_analisis_riesgo"
    }
}

print(f"📁 Base de datos: {DATABASE_NAME}")
print(f"📂 Storage path: {STORAGE_PATH}")

## 3️⃣ Configuración de Spark para Delta Lake

In [ ]:
# ============================================
# CONFIGURACIONES DE SPARK PARA DELTA LAKE
# ============================================

# Habilitar Change Data Feed por defecto para nuevas tablas
spark.conf.set("spark.databricks.delta.properties.defaults.enableChangeDataFeed", "true")

# Habilitar optimizaciones de escritura
spark.conf.set("spark.databricks.delta.optimizeWrite.enabled", "true")

# Habilitar auto-compactación
spark.conf.set("spark.databricks.delta.autoCompact.enabled", "true")

# Configurar retention para historial (7 días)
spark.conf.set("spark.databricks.delta.retentionDurationCheck.enabled", "true")

# Mostrar configuraciones actuales
print("⚙️ Configuraciones de Delta Lake:")
print(f"   CDC habilitado por defecto: {spark.conf.get('spark.databricks.delta.properties.defaults.enableChangeDataFeed')}")
print(f"   Optimize Write: {spark.conf.get('spark.databricks.delta.optimizeWrite.enabled')}")
print(f"   Auto Compact: {spark.conf.get('spark.databricks.delta.autoCompact.enabled')}")

## 4️⃣ Creación de la Base de Datos

In [ ]:
# ============================================
# CREAR BASE DE DATOS
# ============================================

spark.sql(f"""
    CREATE DATABASE IF NOT EXISTS {DATABASE_NAME}
    COMMENT 'Base de datos para proyecto de CDC, SCD y Liquid Clustering con datos financieros'
""")

# Usar la base de datos
spark.sql(f"USE {DATABASE_NAME}")

print(f"✅ Base de datos '{DATABASE_NAME}' creada y seleccionada")

# Verificar
spark.sql("SELECT current_database()").show()

## 5️⃣ Creación de Tablas Bronze con CDC y Liquid Clustering

### 📌 Características importantes:
- **CLUSTER BY**: Habilita Liquid Clustering (reemplaza partitioning + Z-ORDER)
- **enableChangeDataFeed**: Habilita CDC para capturar cambios
- Las columnas de clustering se eligen según patrones de consulta esperados

In [ ]:
# ============================================
# TABLA BRONZE: CLIENTES
# ============================================
# Clustering por: cliente_id (búsquedas frecuentes), fecha_ingesta (time-based queries)

spark.sql(f"""
    CREATE TABLE IF NOT EXISTS {TABLES['bronze']['clientes']} (
        -- Identificadores
        cliente_id STRING NOT NULL COMMENT 'ID único del cliente',
        
        -- Datos personales
        nombre STRING COMMENT 'Nombre completo del cliente',
        email STRING COMMENT 'Correo electrónico',
        telefono STRING COMMENT 'Número de teléfono',
        
        -- Dirección
        direccion STRING COMMENT 'Dirección completa',
        ciudad STRING COMMENT 'Ciudad de residencia',
        pais STRING COMMENT 'País de residencia',
        codigo_postal STRING COMMENT 'Código postal',
        
        -- Datos adicionales
        fecha_nacimiento DATE COMMENT 'Fecha de nacimiento',
        genero STRING COMMENT 'Género del cliente',
        segmento_cliente STRING COMMENT 'Segmento: RETAIL, PREMIUM, VIP, CORPORATE',
        estado STRING COMMENT 'Estado: ACTIVO, INACTIVO, SUSPENDIDO',
        
        -- Metadatos de ingesta
        fecha_registro TIMESTAMP COMMENT 'Fecha de registro en sistema origen',
        fecha_ingesta TIMESTAMP COMMENT 'Timestamp de ingesta a Bronze',
        fuente STRING COMMENT 'Sistema origen de los datos',
        operacion STRING COMMENT 'Tipo de operación: INSERT, UPDATE, DELETE'
    )
    USING DELTA
    CLUSTER BY (cliente_id, fecha_ingesta)
    COMMENT 'Tabla Bronze de Clientes - Raw data con CDC habilitado'
    TBLPROPERTIES (
        'delta.enableChangeDataFeed' = 'true',
        'delta.autoOptimize.optimizeWrite' = 'true',
        'delta.autoOptimize.autoCompact' = 'true',
        'delta.logRetentionDuration' = 'interval 30 days',
        'delta.deletedFileRetentionDuration' = 'interval 7 days'
    )
""")

print(f"✅ Tabla {TABLES['bronze']['clientes']} creada con Liquid Clustering y CDC")

In [ ]:
# ============================================
# TABLA BRONZE: CUENTAS BANCARIAS
# ============================================
# Clustering por: numero_cuenta (PK), cliente_id (FK frecuente)

spark.sql(f"""
    CREATE TABLE IF NOT EXISTS {TABLES['bronze']['cuentas']} (
        -- Identificadores
        numero_cuenta STRING NOT NULL COMMENT 'Número único de cuenta',
        cliente_id STRING NOT NULL COMMENT 'ID del cliente propietario',
        
        -- Información de la cuenta
        tipo_cuenta STRING COMMENT 'Tipo: AHORRO, CORRIENTE, CREDITO, INVERSION',
        moneda STRING COMMENT 'Código de moneda ISO (USD, EUR, PEN, etc.)',
        saldo_actual DECIMAL(18,2) COMMENT 'Saldo actual de la cuenta',
        saldo_disponible DECIMAL(18,2) COMMENT 'Saldo disponible para transacciones',
        limite_credito DECIMAL(18,2) COMMENT 'Límite de crédito (si aplica)',
        
        -- Estado y fechas
        estado STRING COMMENT 'Estado: ACTIVA, BLOQUEADA, CERRADA',
        fecha_apertura DATE COMMENT 'Fecha de apertura de la cuenta',
        fecha_ultimo_movimiento TIMESTAMP COMMENT 'Fecha del último movimiento',
        
        -- Información adicional
        sucursal STRING COMMENT 'Código de sucursal',
        ejecutivo_cuenta STRING COMMENT 'ID del ejecutivo asignado',
        tasa_interes DECIMAL(5,4) COMMENT 'Tasa de interés aplicable',
        
        -- Metadatos de ingesta
        fecha_ingesta TIMESTAMP COMMENT 'Timestamp de ingesta a Bronze',
        fuente STRING COMMENT 'Sistema origen',
        operacion STRING COMMENT 'Tipo de operación: INSERT, UPDATE, DELETE'
    )
    USING DELTA
    CLUSTER BY (numero_cuenta, cliente_id)
    COMMENT 'Tabla Bronze de Cuentas Bancarias - Raw data con CDC habilitado'
    TBLPROPERTIES (
        'delta.enableChangeDataFeed' = 'true',
        'delta.autoOptimize.optimizeWrite' = 'true',
        'delta.autoOptimize.autoCompact' = 'true'
    )
""")

print(f"✅ Tabla {TABLES['bronze']['cuentas']} creada con Liquid Clustering y CDC")

In [ ]:
# ============================================
# TABLA BRONZE: TRANSACCIONES
# ============================================
# Clustering por: fecha_transaccion (time-series), numero_cuenta (joins frecuentes)

spark.sql(f"""
    CREATE TABLE IF NOT EXISTS {TABLES['bronze']['transacciones']} (
        -- Identificadores
        transaccion_id STRING NOT NULL COMMENT 'ID único de la transacción',
        numero_cuenta STRING NOT NULL COMMENT 'Número de cuenta asociada',
        
        -- Detalles de la transacción
        tipo_transaccion STRING COMMENT 'Tipo: DEPOSITO, RETIRO, TRANSFERENCIA, PAGO, COMPRA',
        monto DECIMAL(18,2) COMMENT 'Monto de la transacción',
        moneda STRING COMMENT 'Código de moneda',
        descripcion STRING COMMENT 'Descripción de la transacción',
        
        -- Información temporal
        fecha_transaccion TIMESTAMP COMMENT 'Fecha y hora de la transacción',
        fecha_liquidacion DATE COMMENT 'Fecha de liquidación',
        
        -- Canal y ubicación
        canal STRING COMMENT 'Canal: ATM, ONLINE, MOBILE, SUCURSAL, POS',
        ubicacion STRING COMMENT 'Ubicación de la transacción',
        dispositivo_id STRING COMMENT 'ID del dispositivo (si aplica)',
        
        -- Cuenta destino (para transferencias)
        cuenta_destino STRING COMMENT 'Cuenta destino para transferencias',
        banco_destino STRING COMMENT 'Banco destino para transferencias interbancarias',
        
        -- Estado y categorización
        estado STRING COMMENT 'Estado: PENDIENTE, COMPLETADA, RECHAZADA, REVERSADA',
        categoria STRING COMMENT 'Categoría: ALIMENTOS, TRANSPORTE, SERVICIOS, etc.',
        
        -- Información de riesgo
        score_fraude DECIMAL(5,2) COMMENT 'Score de riesgo de fraude (0-100)',
        requiere_revision BOOLEAN COMMENT 'Flag si requiere revisión manual',
        
        -- Metadatos de ingesta
        fecha_ingesta TIMESTAMP COMMENT 'Timestamp de ingesta a Bronze',
        fuente STRING COMMENT 'Sistema origen',
        operacion STRING COMMENT 'Tipo de operación'
    )
    USING DELTA
    CLUSTER BY (fecha_transaccion, numero_cuenta, tipo_transaccion)
    COMMENT 'Tabla Bronze de Transacciones - Raw data con CDC habilitado'
    TBLPROPERTIES (
        'delta.enableChangeDataFeed' = 'true',
        'delta.autoOptimize.optimizeWrite' = 'true',
        'delta.autoOptimize.autoCompact' = 'true',
        'delta.logRetentionDuration' = 'interval 90 days'
    )
""")

print(f"✅ Tabla {TABLES['bronze']['transacciones']} creada con Liquid Clustering y CDC")

## 6️⃣ Verificación de Tablas Creadas

In [ ]:
# ============================================
# VERIFICAR TABLAS CREADAS
# ============================================

print("📋 Tablas en la base de datos:")
spark.sql(f"SHOW TABLES IN {DATABASE_NAME}").show(truncate=False)

In [ ]:
# Verificar propiedades de Liquid Clustering y CDC
for layer, tables in TABLES.items():
    if layer == "bronze":
        for table_name, table_full_name in tables.items():
            print(f"\n{'='*60}")
            print(f"📊 Detalles de: {table_full_name}")
            print('='*60)
            
            # Mostrar propiedades de la tabla
            spark.sql(f"DESCRIBE DETAIL {table_full_name}").show(vertical=True, truncate=False)

In [ ]:
# Verificar propiedades específicas de la tabla (CDC y Clustering)
print("\n📝 Propiedades de tabla Bronze Clientes:")
spark.sql(f"SHOW TBLPROPERTIES {TABLES['bronze']['clientes']}").show(truncate=False)

## 7️⃣ Generación de Datos Sintéticos

Generaremos datos de prueba para simular un sistema bancario real.

In [ ]:
from pyspark.sql.functions import (
    col, lit, expr, rand, randn, floor, ceil,
    date_add, date_sub, current_timestamp, current_date,
    concat, lpad, when, round as spark_round,
    to_timestamp, to_date, unix_timestamp,
    array, element_at, shuffle
)
from pyspark.sql.types import *
import random

# Configuración de generación de datos
NUM_CLIENTES = 1000
NUM_CUENTAS = 1500  # Algunos clientes tienen múltiples cuentas
NUM_TRANSACCIONES = 10000

print(f"🔄 Generando datos sintéticos:")
print(f"   - Clientes: {NUM_CLIENTES:,}")
print(f"   - Cuentas: {NUM_CUENTAS:,}")
print(f"   - Transacciones: {NUM_TRANSACCIONES:,}")

In [ ]:
# ============================================
# GENERAR DATOS DE CLIENTES
# ============================================

# Arrays para datos aleatorios
nombres = ["Juan", "María", "Carlos", "Ana", "Pedro", "Laura", "Miguel", "Sofia", "Diego", "Valentina",
           "Andrés", "Camila", "Roberto", "Isabella", "Fernando", "Lucía", "Gabriel", "Paula", "Daniel", "Martina"]
apellidos = ["García", "Rodríguez", "Martínez", "López", "González", "Hernández", "Pérez", "Sánchez", 
             "Ramírez", "Torres", "Flores", "Rivera", "Gómez", "Díaz", "Reyes", "Morales", "Ortiz", "Cruz"]
ciudades = ["Lima", "Arequipa", "Trujillo", "Chiclayo", "Piura", "Cusco", "Huancayo", "Iquitos", "Tacna", "Puno"]
segmentos = ["RETAIL", "PREMIUM", "VIP", "CORPORATE"]
estados = ["ACTIVO", "ACTIVO", "ACTIVO", "ACTIVO", "INACTIVO"]  # 80% activos

# Crear DataFrame base
clientes_df = spark.range(1, NUM_CLIENTES + 1).select(
    # ID del cliente
    concat(lit("CLI-"), lpad(col("id").cast("string"), 8, "0")).alias("cliente_id"),
    
    # Nombre completo
    concat(
        element_at(array(*[lit(n) for n in nombres]), (floor(rand() * len(nombres)) + 1).cast("int")),
        lit(" "),
        element_at(array(*[lit(a) for a in apellidos]), (floor(rand() * len(apellidos)) + 1).cast("int")),
        lit(" "),
        element_at(array(*[lit(a) for a in apellidos]), (floor(rand() * len(apellidos)) + 1).cast("int"))
    ).alias("nombre"),
    
    # Email
    concat(
        lit("cliente"),
        col("id").cast("string"),
        lit("@"),
        element_at(array(lit("gmail.com"), lit("hotmail.com"), lit("yahoo.com"), lit("outlook.com")), 
                  (floor(rand() * 4) + 1).cast("int"))
    ).alias("email"),
    
    # Teléfono
    concat(lit("+51 9"), lpad((floor(rand() * 99999999) + 10000000).cast("string"), 8, "0")).alias("telefono"),
    
    # Dirección
    concat(
        lit("Calle "),
        (floor(rand() * 100) + 1).cast("string"),
        lit(" #"),
        (floor(rand() * 999) + 1).cast("string")
    ).alias("direccion"),
    
    # Ciudad
    element_at(array(*[lit(c) for c in ciudades]), (floor(rand() * len(ciudades)) + 1).cast("int")).alias("ciudad"),
    
    # País
    lit("Perú").alias("pais"),
    
    # Código postal
    lpad((floor(rand() * 99999) + 10000).cast("string"), 5, "0").alias("codigo_postal"),
    
    # Fecha de nacimiento (entre 18 y 70 años)
    date_sub(current_date(), (floor(rand() * 52 * 365) + 18 * 365).cast("int")).alias("fecha_nacimiento"),
    
    # Género
    when(rand() < 0.5, lit("M")).otherwise(lit("F")).alias("genero"),
    
    # Segmento
    element_at(array(*[lit(s) for s in segmentos]), (floor(rand() * len(segmentos)) + 1).cast("int")).alias("segmento_cliente"),
    
    # Estado
    element_at(array(*[lit(e) for e in estados]), (floor(rand() * len(estados)) + 1).cast("int")).alias("estado"),
    
    # Fecha de registro (últimos 5 años)
    to_timestamp(date_sub(current_date(), (floor(rand() * 5 * 365)).cast("int"))).alias("fecha_registro"),
    
    # Metadatos de ingesta
    current_timestamp().alias("fecha_ingesta"),
    lit("CORE_BANKING").alias("fuente"),
    lit("INSERT").alias("operacion")
)

print(f"✅ DataFrame de clientes generado: {clientes_df.count()} registros")
clientes_df.show(5, truncate=False)

In [ ]:
# ============================================
# INSERTAR DATOS EN BRONZE_CLIENTES
# ============================================

clientes_df.write \
    .format("delta") \
    .mode("append") \
    .saveAsTable(TABLES['bronze']['clientes'])

print(f"✅ Datos insertados en {TABLES['bronze']['clientes']}")

# Verificar conteo
count = spark.table(TABLES['bronze']['clientes']).count()
print(f"📊 Total de registros: {count:,}")

In [ ]:
# ============================================
# GENERAR DATOS DE CUENTAS BANCARIAS
# ============================================

tipos_cuenta = ["AHORRO", "CORRIENTE", "CREDITO", "INVERSION"]
monedas = ["PEN", "PEN", "PEN", "USD"]  # 75% en soles
estados_cuenta = ["ACTIVA", "ACTIVA", "ACTIVA", "ACTIVA", "BLOQUEADA"]  # 80% activas
sucursales = ["SUC-001", "SUC-002", "SUC-003", "SUC-004", "SUC-005", "SUC-010", "SUC-015", "SUC-020"]

# Obtener IDs de clientes existentes
clientes_ids = spark.table(TABLES['bronze']['clientes']).select("cliente_id").collect()
clientes_list = [row.cliente_id for row in clientes_ids]

cuentas_df = spark.range(1, NUM_CUENTAS + 1).select(
    # Número de cuenta
    concat(lit("191-"), lpad(col("id").cast("string"), 10, "0")).alias("numero_cuenta"),
    
    # Cliente ID (distribuido entre clientes existentes)
    element_at(
        array(*[lit(c) for c in clientes_list]),
        (floor(rand() * len(clientes_list)) + 1).cast("int")
    ).alias("cliente_id"),
    
    # Tipo de cuenta
    element_at(array(*[lit(t) for t in tipos_cuenta]), (floor(rand() * len(tipos_cuenta)) + 1).cast("int")).alias("tipo_cuenta"),
    
    # Moneda
    element_at(array(*[lit(m) for m in monedas]), (floor(rand() * len(monedas)) + 1).cast("int")).alias("moneda"),
    
    # Saldo actual (entre 0 y 500,000)
    spark_round(rand() * 500000, 2).alias("saldo_actual"),
    
    # Saldo disponible (90-100% del saldo actual)
    spark_round(rand() * 500000 * (0.9 + rand() * 0.1), 2).alias("saldo_disponible"),
    
    # Límite de crédito (solo para cuentas de crédito)
    when(
        element_at(array(*[lit(t) for t in tipos_cuenta]), (floor(rand() * len(tipos_cuenta)) + 1).cast("int")) == "CREDITO",
        spark_round(rand() * 50000 + 5000, 2)
    ).otherwise(lit(None)).alias("limite_credito"),
    
    # Estado
    element_at(array(*[lit(e) for e in estados_cuenta]), (floor(rand() * len(estados_cuenta)) + 1).cast("int")).alias("estado"),
    
    # Fecha de apertura
    date_sub(current_date(), (floor(rand() * 3 * 365)).cast("int")).alias("fecha_apertura"),
    
    # Último movimiento
    to_timestamp(date_sub(current_date(), (floor(rand() * 30)).cast("int"))).alias("fecha_ultimo_movimiento"),
    
    # Sucursal
    element_at(array(*[lit(s) for s in sucursales]), (floor(rand() * len(sucursales)) + 1).cast("int")).alias("sucursal"),
    
    # Ejecutivo
    concat(lit("EJE-"), lpad((floor(rand() * 50) + 1).cast("string"), 3, "0")).alias("ejecutivo_cuenta"),
    
    # Tasa de interés (1-15%)
    spark_round(rand() * 0.14 + 0.01, 4).alias("tasa_interes"),
    
    # Metadatos
    current_timestamp().alias("fecha_ingesta"),
    lit("CORE_BANKING").alias("fuente"),
    lit("INSERT").alias("operacion")
)

# Insertar en tabla Bronze
cuentas_df.write \
    .format("delta") \
    .mode("append") \
    .saveAsTable(TABLES['bronze']['cuentas'])

print(f"✅ Datos insertados en {TABLES['bronze']['cuentas']}")
print(f"📊 Total de registros: {spark.table(TABLES['bronze']['cuentas']).count():,}")

In [ ]:
# ============================================
# GENERAR DATOS DE TRANSACCIONES
# ============================================

tipos_transaccion = ["DEPOSITO", "RETIRO", "TRANSFERENCIA", "PAGO", "COMPRA"]
canales = ["ATM", "ONLINE", "MOBILE", "SUCURSAL", "POS"]
estados_tx = ["COMPLETADA", "COMPLETADA", "COMPLETADA", "COMPLETADA", "PENDIENTE", "RECHAZADA"]
categorias = ["ALIMENTOS", "TRANSPORTE", "SERVICIOS", "ENTRETENIMIENTO", "SALUD", "EDUCACION", "OTROS"]

# Obtener cuentas existentes
cuentas_ids = spark.table(TABLES['bronze']['cuentas']).select("numero_cuenta").collect()
cuentas_list = [row.numero_cuenta for row in cuentas_ids]

transacciones_df = spark.range(1, NUM_TRANSACCIONES + 1).select(
    # ID de transacción
    concat(lit("TXN-"), lpad(col("id").cast("string"), 12, "0")).alias("transaccion_id"),
    
    # Cuenta asociada
    element_at(
        array(*[lit(c) for c in cuentas_list]),
        (floor(rand() * len(cuentas_list)) + 1).cast("int")
    ).alias("numero_cuenta"),
    
    # Tipo de transacción
    element_at(array(*[lit(t) for t in tipos_transaccion]), (floor(rand() * len(tipos_transaccion)) + 1).cast("int")).alias("tipo_transaccion"),
    
    # Monto (entre 1 y 50,000)
    spark_round(rand() * 49999 + 1, 2).alias("monto"),
    
    # Moneda
    element_at(array(lit("PEN"), lit("PEN"), lit("PEN"), lit("USD")), (floor(rand() * 4) + 1).cast("int")).alias("moneda"),
    
    # Descripción
    concat(
        lit("Transacción #"),
        col("id").cast("string")
    ).alias("descripcion"),
    
    # Fecha de transacción (últimos 90 días)
    to_timestamp(
        concat(
            date_sub(current_date(), (floor(rand() * 90)).cast("int")).cast("string"),
            lit(" "),
            lpad((floor(rand() * 24)).cast("string"), 2, "0"),
            lit(":"),
            lpad((floor(rand() * 60)).cast("string"), 2, "0"),
            lit(":"),
            lpad((floor(rand() * 60)).cast("string"), 2, "0")
        )
    ).alias("fecha_transaccion"),
    
    # Fecha de liquidación
    date_sub(current_date(), (floor(rand() * 88)).cast("int")).alias("fecha_liquidacion"),
    
    # Canal
    element_at(array(*[lit(c) for c in canales]), (floor(rand() * len(canales)) + 1).cast("int")).alias("canal"),
    
    # Ubicación
    element_at(array(*[lit(c) for c in ciudades]), (floor(rand() * len(ciudades)) + 1).cast("int")).alias("ubicacion"),
    
    # Dispositivo
    concat(lit("DEV-"), lpad((floor(rand() * 10000)).cast("string"), 6, "0")).alias("dispositivo_id"),
    
    # Cuenta destino (para transferencias)
    when(rand() < 0.2, 
         concat(lit("191-"), lpad((floor(rand() * 9999999999) + 1).cast("string"), 10, "0"))
    ).otherwise(lit(None)).alias("cuenta_destino"),
    
    # Banco destino
    when(rand() < 0.1,
         element_at(array(lit("BCP"), lit("BBVA"), lit("INTERBANK"), lit("SCOTIABANK")), (floor(rand() * 4) + 1).cast("int"))
    ).otherwise(lit(None)).alias("banco_destino"),
    
    # Estado
    element_at(array(*[lit(e) for e in estados_tx]), (floor(rand() * len(estados_tx)) + 1).cast("int")).alias("estado"),
    
    # Categoría
    element_at(array(*[lit(c) for c in categorias]), (floor(rand() * len(categorias)) + 1).cast("int")).alias("categoria"),
    
    # Score de fraude (0-100)
    spark_round(rand() * 100, 2).alias("score_fraude"),
    
    # Requiere revisión (si score > 80)
    (rand() * 100 > 80).alias("requiere_revision"),
    
    # Metadatos
    current_timestamp().alias("fecha_ingesta"),
    lit("TRANSACTION_SYSTEM").alias("fuente"),
    lit("INSERT").alias("operacion")
)

# Insertar en tabla Bronze
transacciones_df.write \
    .format("delta") \
    .mode("append") \
    .saveAsTable(TABLES['bronze']['transacciones'])

print(f"✅ Datos insertados en {TABLES['bronze']['transacciones']}")
print(f"📊 Total de registros: {spark.table(TABLES['bronze']['transacciones']).count():,}")

## 8️⃣ Verificación Final del Setup

In [ ]:
# ============================================
# RESUMEN DEL SETUP
# ============================================

print("\n" + "="*70)
print("📊 RESUMEN DEL SETUP")
print("="*70)

for table_name in [TABLES['bronze']['clientes'], TABLES['bronze']['cuentas'], TABLES['bronze']['transacciones']]:
    count = spark.table(table_name).count()
    print(f"\n📁 {table_name}:")
    print(f"   └─ Registros: {count:,}")
    
    # Verificar CDC
    props = spark.sql(f"SHOW TBLPROPERTIES {table_name}").filter("key = 'delta.enableChangeDataFeed'").collect()
    if props:
        print(f"   └─ CDC habilitado: {props[0]['value']}")
    
    # Verificar Clustering
    detail = spark.sql(f"DESCRIBE DETAIL {table_name}").collect()[0]
    if detail.clusteringColumns:
        print(f"   └─ Liquid Clustering: {detail.clusteringColumns}")

In [ ]:
# Verificar que CDC está capturando cambios (historial)
print("\n📜 Historial de cambios en Bronze Clientes:")
spark.sql(f"DESCRIBE HISTORY {TABLES['bronze']['clientes']} LIMIT 5").show(truncate=False)

---

## ✅ Setup Completado

### Lo que hemos configurado:

1. ✅ Base de datos `financial_lakehouse` creada
2. ✅ Tablas Bronze con **Liquid Clustering** habilitado
3. ✅ **Change Data Feed (CDC)** habilitado en todas las tablas
4. ✅ Datos sintéticos cargados para pruebas

### Próximo paso:
Continuar con el **Notebook 02: Bronze Layer - CDC** para ver cómo capturar y procesar cambios incrementales.

---